In [11]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from pathlib import Path
import json
import matplotlib.image as mpimg 
from PIL import Image
from random import randint
import torch
from torch.utils.data import Dataset, DataLoader
import sklearn
import sklearn.metrics
from sklearn.metrics import accuracy_score
import torch.nn as nn
import math
import torch.nn.functional as F
import copy
from ignite.metrics import DiceCoefficient, ConfusionMatrix
from monai.losses import DiceCELoss
from monai.networks.nets import UNETR
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
)
from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)
from tqdm import tqdm
import os
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from sklearn.model_selection import train_test_split

In [12]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataloader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)

class DataTrain(Dataset):

    def __init__(self, data, annotation):
        self.traininputtensor = torch.tensor(data, dtype=torch.float)
        self.output = torch.tensor(annotation, dtype=torch.float)
    
    def __getitem__(self, index):
        input_image = self.traininputtensor[index].unsqueeze(0)  # Добавляет измерение канала, становится (1, depth, height, width)
        output_label = self.output[index].unsqueeze(0)  # То же самое для метки
        return input_image, output_label

    def __len__(self):
        return self.traininputtensor.size(dim=0)


class DataTest(Dataset):

    def __init__(self, data, annotation):
        self.testinputtensor = torch.tensor(data, dtype=torch.float)
        self.output = torch.tensor(annotation, dtype=torch.float)

    def __getitem__(self, index):
        input_image = self.testinputtensor[index].unsqueeze(0)  # Добавляет измерение канала, становится (1, depth, height, width)
        output_label = self.output[index].unsqueeze(0)  # То же самое для метки
        return input_image, output_label

    def __len__(self):
        return self.testinputtensor.size(dim=0)


def changezax(data, z):
    while len(data) < z:
        data.append(np.zeros((96, 96), dtype=np.uint8))
    data = data[:z]
    return data


def splitToTest1(annot, data, test=3):
    k = test
    traindata = data.copy()
    trainannot = annot.copy()
    testdata = []
    testannot = []
    testdatafinal = []
    testannotfinal = []
    trainannotfinal = []
    traindatafinal = []
    while k!=0:
        rand = randint(0, len(traindata)-1)
        testdata.append(traindata[rand])
        testannot.append(trainannot[rand])
        traindata.pop(rand - 1)
        trainannot.pop(rand - 1)
        k -= 1
    for i in range(len(testdata)):
        testdatafinal.append(np.array(testdata[i]))
        print(np.array(testdata[i]).shape)
        testannotfinal.append(np.array(testannot[i]))
    for k in range(len(traindata)):
        traindatafinal.append(np.array(traindata[k]))
        trainannotfinal.append(np.array(trainannot[k]))
    
    return[testdatafinal, testannotfinal, traindatafinal, trainannotfinal]




def splitToTest(annot, data, test=3):
    k = test
    traindata = data.copy()
    trainannot = annot.copy()
    testdata = []
    testannot = []
    data1 = []
    dat = []
    while k!=0:
        rand = randint(0, len(traindata))
        testdata.append(traindata[rand - 1])
        testannot.append(trainannot[rand - 1])
        traindata.pop(rand - 1)
        trainannot.pop(rand - 1)
        k -= 1
    for i in range(len(traindata)):
        data1.append({"image": torch.tensor((traindata[i])), "label": torch.tensor(trainannot[i])})
    for k in range(len(testdata)):
        dat.append({"image": torch.tensor(testdata[k]), "label": torch.tensor(testannot[k])})
    return[data1, dat]


def preperingdata(path_to_masks, path_to_img, z):
    labels = []
    data = []
    for floader in path_to_masks.iterdir():
            print("Floader", floader)
            annot = []
            for file in floader.iterdir():
                img = Image.open(file)
                if img.mode != "L":
                    img = img.convert("L")
                img = img.resize((96, 96))
                img_array = np.array(img)
                annot.append(img_array)
            #print(np.array(changezax(annot, z=z)).shape)
            labels.append(changezax(annot, z=z))
    for floader1 in path_to_img.iterdir():
        for floader2 in floader1.iterdir():
            if floader2.is_dir():
                pic = []
                print("Floader", floader2)
                for picture in floader2.iterdir():
                    imga = Image.open(picture)
                    if imga.mode != "L":
                        imga = imga.convert("L")
                    imga = imga.resize((96, 96))
                    array = np.array(imga)
                    pic.append(array)
                data.append(changezax(pic, z=z))
                #print(np.array(pic).shape)
    return [data, labels]

In [13]:
directory_path = Path(r'C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\workspase')
mask_path = Path(r'C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks')
data = preperingdata(mask_path, directory_path, z=96)


data1 = splitToTest(data=data[0], annot=data[1])


# train_ds = CacheDataset(
#     data=data1[0],
#     cache_num=24,
#     cache_rate=1.0,
#     num_workers=8,
# )
train_loader = DataLoader(data1[0], batch_size=1, shuffle=True, num_workers=8, pin_memory=True)
# val_ds = CacheDataset(
#     data=data1[1], 
#     cache_num=6, 
#     cache_rate=1.0, 
#     num_workers=4
# )
val_loader = DataLoader(data1[1], batch_size=1, shuffle=False, num_workers=4, pin_memory=True)
num_epochs = 20
rate_learning = 1e-4
device = get_default_device()
model = UNETR(
    in_channels=1,
    out_channels=1,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)
modelwise = UNETR(   #just_to_copy_weights
    in_channels=1,
    out_channels=14,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)
modelwise.load_state_dict(torch.load(os.path.join(r"D:\UNETR", "best_metric_model.pth")))
model_state_dict1 = modelwise.state_dict()
for name_dst, param_dst in model.named_parameters():
    if name_dst in modelwise.state_dict():
        param_src = model.state_dict()[name_dst]
        if param_src.size() == param_dst.size():
            param_dst.data.copy_(param_src.data)
        else:
            print(f"Skipping layer {name_dst} due to size mismatch")
        
loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
torch.backends.cudnn.benchmark = True
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\1
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\10
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\11
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\12
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\13
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\14
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\15
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\16
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\17
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\18
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\19
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\2
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\20
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat_backup\masks\3
Floader C:\Users\User\Desktop\pilsen_pigs_2023_cvat

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\monai\utils\deprecate_utils.py:221: FutureWarning: monai.networks.nets.unetr UNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)


In [ ]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-175,
            a_max=250,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=4,
            image_key="image",
            image_threshold=0,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[0],
            prob=0.10,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[1],
            prob=0.10,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[2],
            prob=0.10,
        ),
        RandRotate90d(
            keys=["image", "label"],
            prob=0.10,
            max_k=3,
        ),
        RandShiftIntensityd(
            keys=["image"],
            offsets=0.10,
            prob=0.50,
        ),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        ScaleIntensityRanged(keys=["image"], a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
        CropForegroundd(keys=["image", "label"], source_key="image"),
    ]
)


In [6]:

def dice_coefficient_3d(y_true, y_pred, smooth=1.0):
    # Плоское представление массивов
    y_true_f = y_true.contiguous().view(-1)
    y_pred_f = y_pred.contiguous().view(-1)
    # Вычисление пересечения
    intersection = (y_true_f * y_pred_f).sum()
    # Расчёт Dice Coefficient
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)

# Предположим, у вас есть 3D маски
y_true = torch.rand(size=(1, 1, 10, 256, 256))  # Истинная маска
y_pred = torch.rand(size=(1, 1, 10, 256, 256))  # Предсказанная маска

# Binarize predictions and ground truth for example
y_true = (y_true > 0.5).float()
y_pred = (y_pred > 0.5).float()

dice_score = dice_coefficient_3d(y_true, y_pred)
print(f"Dice Score: {dice_score}")

#output = model(t)
#print(output)


Dice Score: 0.5001701712608337


In [4]:
def validation(epoch_iterator_val):
    model.eval()
    with torch.no_grad():
        for batch in epoch_iterator_val:
            val_inputs, val_labels = (batch["image"].cuda(), batch["label"].cuda())
            val_outputs = sliding_window_inference(val_inputs, (512 ,512, 784), 4, model)
            val_labels_list = decollate_batch(val_labels)
            val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]
            val_outputs_list = decollate_batch(val_outputs)
            val_output_convert = [post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list]
            dice_metric(y_pred=val_output_convert, y=val_labels_convert)
            epoch_iterator_val.set_description("Validate (%d / %d Steps)" % (global_step, 10.0))  # noqa: B038
        mean_dice_val = dice_metric.aggregate().item()
        dice_metric.reset()
    return mean_dice_val


def train(global_step, train_loader, dice_val_best, global_step_best):
    model.train()
    epoch_loss = 0
    step = 0
    epoch_iterator = tqdm(train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True)
    for step, batch in enumerate(epoch_iterator):
        step += 1
        x, y = (batch["image"].cuda(), batch["label"].cuda())
        logit_map = model(x)
        loss = loss_function(logit_map, y)
        loss.backward()
        epoch_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        epoch_iterator.set_description(  # noqa: B038
            "Training (%d / %d Steps) (loss=%2.5f)" % (global_step, max_iterations, loss)
        )
        if (global_step % eval_num == 0 and global_step != 0) or global_step == max_iterations:
            epoch_iterator_val = tqdm(val_loader, desc="Validate (X / X Steps) (dice=X.X)", dynamic_ncols=True)
            dice_val = validation(epoch_iterator_val)
            epoch_loss /= step
            epoch_loss_values.append(epoch_loss)
            metric_values.append(dice_val)
            if dice_val > dice_val_best:
                dice_val_best = dice_val
                global_step_best = global_step
                torch.save(model.state_dict(), os.path.join(r"D:\result", "best_metric_model.pth"))
                print(
                    "Model Was Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(dice_val_best, dice_val)
                )
            else:
                print(
                    "Model Was Not Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(
                        dice_val_best, dice_val
                    )
                )
        global_step += 1
    return global_step, dice_val_best, global_step_best


In [4]:
max_iterations = 100
eval_num = 500
post_label = AsDiscrete(to_onehot=1)
post_pred = AsDiscrete(argmax=True, to_onehot=1)
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
global_step = 0
dice_val_best = 0.0
global_step_best = 0
epoch_loss_values = []
metric_values = []
while global_step < max_iterations:
    global_step, dice_val_best, global_step_best = train(global_step, train_loader, dice_val_best, global_step_best)
model.load_state_dict(torch.load(os.path.join(r"D:\result", "best_metric_model.pth")))




plt.figure("train", (12, 6))
plt.subplot(1, 2, 1)
plt.title("Iteration Average Loss")
x = [eval_num * (i + 1) for i in range(len(epoch_loss_values))]
y = epoch_loss_values
plt.xlabel("Iteration")
plt.plot(x, y)
plt.subplot(1, 2, 2)
plt.title("Val Mean Dice")
x = [eval_num * (i + 1) for i in range(len(metric_values))]
y = metric_values
plt.xlabel("Iteration")
plt.plot(x, y)
plt.show()

NameError: name 'train' is not defined

In [14]:
def dice_coefficient_3d(y_true, y_pred, smooth=1.0):
    # Плоское представление массивов
    y_true_f = y_true.contiguous().view(-1)
    y_pred_f = y_pred.contiguous().view(-1)
    # Вычисление пересечения
    intersection = (y_true_f * y_pred_f).sum()
    # Расчёт Dice Coefficient
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)





data2 = splitToTest1(data=data[0], annot=data[1])
train = DataTrain(data2[2], data2[3])
test = DataTest(data2[0], data2[1])
dataloader_train = DataLoader(dataset=train, batch_size=1, shuffle=True)
dataloader_test = DataLoader(dataset=test, batch_size=1, shuffle=True)




loss_fn =  DiceCELoss(to_onehot_y=True, softmax=True)
lr = 1e-4
num_epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimazer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
cm = ConfusionMatrix(num_classes=2)
dice = DiceCoefficient(cm)
history = []
history1 = []
acc_val = []
acc_test = []
acc_train = []
for epochs in range(num_epochs):
    with tqdm(total=17, desc=f'Epoch {epochs + 1}/{num_epochs}', unit='batch') as pbar:
        running_loss = 0
        val_loss = 0
        model.train()
        for i, data in enumerate(dataloader_train):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimazer.zero_grad()
            output = model(inputs)
            loss = loss_fn(output, labels)
            loss.backward()
            optimazer.step()
            running_loss += loss.item()
            preds = (labels>0.5).float()
            output = (output>0.5).float()
            acc_train.append(dice_coefficient_3d(preds, output))
            pbar.update(1)
        history.append(running_loss)
correct = 0
total = 0
for j, data in enumerate(dataloader_test):
    with torch.no_grad():
        inputs_for_test, labels_for_test = data
        inputs_for_test = inputs_for_test.to(device)
        labels_for_test = labels_for_test.to(device)
        output_for_test = model(inputs_for_test)
        preds = (labels_for_test>0.5).float()
        output = (output_for_test>0.5).float()
        acc_val.append(dice_coefficient_3d(preds, output))
print("Train history", history)
#print("Validation hisoty", history1)
print("Accuracy train", acc_train)
print("Accuracy validation", acc_val)

(96, 96, 96)
(96, 96, 96)
(96, 96, 96)


Epoch 1/2:   0%|          | 0/17 [00:00<?, ?batch/s]C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\monai\losses\dice.py:146: UserWarning: single channel prediction, `softmax=True` ignored.
  warnings.warn("single channel prediction, `softmax=True` ignored.")
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\monai\losses\dice.py:155: UserWarning: single channel prediction, `to_onehot_y=True` ignored.
  warnings.warn("single channel prediction, `to_onehot_y=True` ignored.")
Epoch 2/2: 100%|██████████| 17/17 [00:25<00:00,  1.50s/batch]


Train history [28.71602153778076, 24.142449975013733]
Accuracy train [tensor(4.8443e-05, device='cuda:0'), tensor(0.1364, device='cuda:0'), tensor(4.9995e-05, device='cuda:0'), tensor(3.1705e-05, device='cuda:0'), tensor(3.1823e-05, device='cuda:0'), tensor(3.6761e-05, device='cuda:0'), tensor(2.1911e-05, device='cuda:0'), tensor(2.0425e-05, device='cuda:0'), tensor(3.3533e-05, device='cuda:0'), tensor(2.0422e-05, device='cuda:0'), tensor(1.8949e-05, device='cuda:0'), tensor(2.8228e-05, device='cuda:0'), tensor(2.2847e-05, device='cuda:0'), tensor(1.8499e-05, device='cuda:0'), tensor(2.2329e-05, device='cuda:0'), tensor(2.1232e-05, device='cuda:0'), tensor(1.7821e-05, device='cuda:0'), tensor(1.7279e-05, device='cuda:0'), tensor(2.8773e-05, device='cuda:0'), tensor(1.8002e-05, device='cuda:0'), tensor(1.9257e-05, device='cuda:0'), tensor(2.0964e-05, device='cuda:0'), tensor(2.3679e-05, device='cuda:0'), tensor(3.2853e-05, device='cuda:0'), tensor(0.2716, device='cuda:0'), tensor(3.2099